In [75]:
import pandas as pd
import numpy as np


In [76]:
# Data

df = pd.read_excel("../data/ltcm_exhibits_data.xlsx", sheet_name= 'Exhibit 2', skiprows=2)
df = df.dropna(how='any')
df = df.set_index('Unnamed: 0')
df.index = pd.to_datetime(df.index)
df.index.name = 'Date'
df.head()


c:\Users\User\AppData\Local\Programs\Python\Python313\Lib\site-packages\pandas\core\indexes\base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


,Fund Capital ($billions),Gross Monthly Performancea,Net Monthly Performanceb,Index of Net Performance
Date,,,,
1994-03-01,1.1,-0.011,-0.013,0.99
1994-04-01,1.1,0.014,0.008,1.00
1994-05-01,1.2,0.068,0.053,1.05
1994-06-01,1.2,-0.039,-0.029,1.02
1994-07-01,1.4,0.116,0.084,1.10


In [77]:
spy_df = pd.read_excel("../data/spy_data.xlsx", sheet_name= 'excess returns')
spy_df = spy_df.dropna(how='any')
spy_df = spy_df.set_index('date')
spy_df.index = pd.to_datetime(spy_df.index)
spy_df.index.name = 'Date'
spy_df.head()

,SPY
Date,
1994-01-31,0.032401
1994-02-28,-0.031964
1994-03-31,-0.050288
1994-04-30,0.007996
1994-05-31,0.012464


In [78]:
# 2.1

ltcm_gross = df["Gross Monthly Performancea"]  
ltcm_net   = df["Net Monthly Performanceb"] 
spy_excess = spy_df["SPY"]

In [79]:
import statsmodels.api as sm

##　Ｅｘｃｅｒｃｉｃｅ

In [80]:
import pandas as pd
import numpy as np

In [81]:
df = pd.read_excel('../data/spx_data_weekly.xlsx',sheet_name="spx data", header=[0,1], index_col=0)
df.index = pd.to_datetime(df.index)
valid_tickers = []
min_weeks = 52 * 5
for ticker in df.columns.levels[0]:
    if ticker not in df.columns: 
        continue
    if df[ticker].dropna().shape[0] >= min_weeks:
        valid_tickers.append(ticker)

df_filtered = df[valid_tickers]
df.head(), df_filtered.shape

(                  A                              AAPL                  \
             PX_LAST EQY_DVD_YLD_IND PE_RATIO  PX_LAST EQY_DVD_YLD_IND   
 date                                                                    
 2015-07-03  36.4044          1.0988  27.4720  28.2589          1.8401   
 2015-07-10  36.2388          1.1038  27.3471  27.5527          1.8873   
 2015-07-17  36.7447          1.0886  27.7288  28.9696          1.7950   
 2015-07-24  36.1560          1.1063  27.2846  27.8253          1.8688   
 2015-07-31  37.6645          1.0620  26.6192  27.1101          1.9181   
 
                         ABBV                             ABNB  ...      YUM  \
            PE_RATIO  PX_LAST EQY_DVD_YLD_IND PE_RATIO PX_LAST  ... PE_RATIO   
 date                                                           ...            
 2015-07-03  13.0592  44.2648          4.6086  11.9239     NaN  ...  18.6689   
 2015-07-10  12.7328  44.9267          4.5407  12.1022     NaN  ...  18.5282   
 2015-

In [82]:
# 1.1
def get_dividend_yield_extremes(df):
    div_yld = df.xs("EQY_DVD_YLD_IND", axis=1, level=1)
    max_yield = div_yld.idxmax(axis=1)  
    min_yield = div_yld.idxmin(axis=1) 
    return pd.DataFrame({'max_yield_ticker': max_yield, 'min_yield_ticker': min_yield})

extremes_df = get_dividend_yield_extremes(df_filtered)
print(extremes_df.tail())

           max_yield_ticker min_yield_ticker
date                                        
2025-05-30              DOW             NVDA
2025-06-06              DOW             NVDA
2025-06-13              DOW             NVDA
2025-06-20              DOW             NVDA
2025-06-27              DOW             NVDA


In [83]:
def get_average_dividend_yield(df, n_weeks=52):
    div_yld = df.xs("EQY_DVD_YLD_IND", axis=1, level=1).tail(n_weeks)
    mean_yield = div_yld.mean()
    max_ticker = mean_yield.idxmax()
    min_ticker = mean_yield.idxmin()
    return max_ticker, mean_yield[max_ticker], min_ticker, mean_yield[min_ticker]

high, high_val, low, low_val = get_average_dividend_yield(df_filtered)
print(f"Highest average dividend ratio：{high}（{high_val:.2f}%）")
print(f"Lowest average dividend ratio：{low}（{low_val:.2f}%）")

Highest average dividend ratio：MO（7.95%）
Lowest average dividend ratio：NVDA（0.03%）


<span style="color: blue;">

ANS: 

Over the past year, MO had the highest average dividend yield at 7.95%, while NVDA had the lowest at just 0.03%.

The elevated yield for MO was primarily due to its consistently high dividend payouts and relatively stable or modestly declining stock price, indicating a strong commitment to income-focused investors. In contrast, NVDA's extremely low yield reflects its minimal dividend distribution and rapidly rising stock price, characteristic of a high-growth company that reinvests earnings rather than returning capital to shareholders.

Overall, the variation in yields appears to be driven more by differences in dividend policy (D) than by changes in price (P), especially in the case of NVDA.

In [84]:
# 1.2

# --- Extract dividend yield and price ---
dvd_yld = df_filtered.xs("EQY_DVD_YLD_IND", axis=1, level=1)
price = df_filtered.xs("PX_LAST", axis=1, level=1)

# --- Weekly returns ---
ret = price.pct_change().shift(-1)

# --- Ranking each week ---
rank = dvd_yld.rank(axis=1, ascending=False)

n = rank.shape[1]
top_n = int(np.floor(0.2 * n))

# --- Long-only weights (equal 0.01) ---
weights_long = (rank <= top_n).astype(float) * 0.01

# --- Portfolio return ---
carry_ret = (weights_long * ret).sum(axis=1)
carry_ret = carry_ret.dropna()
carry_ret.head()




date
2015-07-03    0.005380
2015-07-10    0.004836
2015-07-17   -0.018145
2015-07-24    0.017742
2015-07-31   -0.005158
dtype: float64

In [85]:
# 1.3
# --- Long-Short weights ---
weights_ls = pd.DataFrame(0, index=rank.index, columns=rank.columns)

weights_ls[rank <= top_n] = 0.01
weights_ls[rank > (n - top_n)] = -0.01

# --- Long-short portfolio return ---
ls_ret = (weights_ls * ret).sum(axis=1)
ls_ret = ls_ret.dropna()
ls_ret.head()

C:\Users\User\AppData\Local\Temp\ipykernel_18224\1727887067.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weights_ls[rank <= top_n] = 0.01
C:\Users\User\AppData\Local\Temp\ipykernel_18224\1727887067.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.01' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  weights_ls[rank > (n - top_n)] = -0.01


date
2015-07-03    0.005102
2015-07-10    0.000766
2015-07-17   -0.009259
2015-07-24    0.009111
2015-07-31   -0.002786
dtype: float64

In [86]:
# 1.4

def perf_stats(r):
    ann_mean = r.mean() * 52
    ann_vol = r.std() * np.sqrt(52)
    sharpe = ann_mean / ann_vol
    
    skew = r.skew()
    var_5 = r.quantile(0.05)
    cvar_5 = r[r <= var_5].mean()
    
    cum = (1 + r).cumprod()
    dd = cum / cum.cummax() - 1
    max_dd = dd.min()
    
    return pd.Series({
        "Mean": ann_mean,
        "Vol": ann_vol,
        "Sharpe": sharpe,
        "Skew": skew,
        "VaR_5%": var_5,
        "CVaR_5%": cvar_5,
        "MaxDD": max_dd
    })

df_add = pd.read_excel(
    '../data/spx_data_weekly.xlsx',
    sheet_name='additional data',
    header=[0,1],
    index_col=0
)

df_add.index = pd.to_datetime(df_add.index)
spy_price = df_add.xs("SPY", axis=1, level=0).xs("PX_LAST", axis=1)
spy_ret = spy_price.pct_change().shift(-1).dropna()
common_idx = carry_ret.index.intersection(spy_ret.index)

carry_ret = carry_ret.loc[common_idx]
ls_ret = ls_ret.loc[common_idx]
spy_ret = spy_ret.loc[common_idx]

perf_table = pd.concat([
    perf_stats(carry_ret),
    perf_stats(ls_ret),
    perf_stats(spy_ret)
], axis=1)

perf_table.columns = ["Carry Long", "Carry Long-Short", "SPY"]
print(perf_table)

         Carry Long  Carry Long-Short       SPY
Mean       0.112670          0.021356  0.140826
Vol        0.160823          0.095895  0.173193
Sharpe     0.700582          0.222706  0.813116
Skew       0.285957          1.559453 -0.595027
VaR_5%    -0.028205         -0.018282 -0.033571
CVaR_5%   -0.049354         -0.026737 -0.056766
MaxDD     -0.347861         -0.179768 -0.318291


In [87]:
#2

import pandas as pd
import numpy as np
import statsmodels.api as sm

def run_lfd(Y, X, freq=52):
    X = sm.add_constant(X)
    model = sm.OLS(Y, X, missing="drop").fit()

    alpha_ann = model.params["const"] * freq
    beta = model.params.drop("const")
    r2 = model.rsquared

    return alpha_ann, beta, r2, model


In [88]:

sector_px = pd.read_excel(
    "../data/spx_data_weekly.xlsx",   # ← 你的 sector excel
    sheet_name="sector data",
    header=[0,1],
    index_col=0,
    parse_dates=True
)

sector_px.index.name = "date"


In [89]:
valid_sectors = [
    'XLK','XLI','XLF','XLC','XLRE','XLE','XLY',
    'XLB','XLV','XLU','XLP'
]

sector_price = sector_px.loc[:, (valid_sectors, "PX_LAST")]

sector_ret = sector_price.pct_change()
sector_ret.columns = sector_ret.columns.get_level_values(0)

# 與策略報酬對齊
sector_ret = sector_ret.loc[common_idx]
sector_ret.head()

,XLK,XLI,XLF,XLC,XLRE,XLE,XLY,XLB,XLV,XLU,XLP
date,,,,,,,,,,,
2015-07-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-07-10,-0.007187,-0.001108,0.000000,NaN,NaN,-0.013954,0.006087,-0.016698,0.005879,0.017523,0.020965
2015-07-17,0.048262,0.013846,0.030434,NaN,NaN,-0.014288,0.019056,-0.000839,0.022450,0.008842,0.018099
2015-07-24,-0.024863,-0.036781,-0.010633,NaN,NaN,-0.040447,-0.003790,-0.054133,-0.027543,-0.023529,-0.007789
2015-07-31,0.005194,0.024952,0.003586,NaN,NaN,-0.001870,0.017374,0.019298,0.023246,0.039216,0.012681


In [90]:
# 2.1

alpha_lo, beta_lo, r2_lo, m_lo = run_lfd(carry_ret, spy_ret.to_frame("SPY"))
alpha_ls, beta_ls, r2_ls, m_ls = run_lfd(ls_ret, spy_ret.to_frame("SPY"))

market_summary = pd.DataFrame({
    "LO": [alpha_lo, beta_lo.iloc[0], r2_lo],
    "LS": [alpha_ls, beta_ls.iloc[0], r2_ls]
}, index=["Alpha (ann.)", "Beta vs SPY", "R²"])

market_summary


,LO,LS
Alpha (ann.),0.006958,0.011508
Beta vs SPY,0.750660,0.069934
R²,0.653506,0.015953


In [91]:
corr_df = pd.concat([
    carry_ret.rename("LO"),
    ls_ret.rename("LS"),
    spy_ret.rename("SPY")
], axis=1)

corr_matrix = corr_df.corr()
corr_matrix


,LO,LS,SPY
LO,1.000000,0.643840,0.808397
LS,0.643840,1.000000,0.126306
SPY,0.808397,0.126306,1.000000


<span style="color: blue;">

ANS:

LO and LS strategies compare to SPY don't have strong alpha.LO has a high beta while LS has a low beta, which is not suprising.

In [92]:
# 2.2
additional_px = pd.read_excel(
    "../data/spx_data_weekly.xlsx",
    sheet_name="additional data",
    header=[0,1],
    index_col=0,
    parse_dates=True
)

additional_px.index.name = "date"
additional_px.head()

,SPY,IEF,IYR,GLD,USO,XBTUSD BGN Curncy,SHV
,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST,PX_LAST
date,,,,,,,
2015-07-03,174.2734,83.7708,52.7722,111.76,151.44,255.22,90.0731
2015-07-10,174.4163,83.7468,53.6596,111.49,141.36,287.13,90.0731
2015-07-17,178.6195,84.0830,54.1105,108.65,135.92,278.27,90.0812
2015-07-24,174.8534,84.6752,53.7905,105.35,128.24,288.16,90.0731
2015-07-31,176.9550,85.1874,54.4306,104.93,124.72,284.69,90.0567


In [93]:
add_price = additional_px.xs("PX_LAST", axis=1, level=1)
factor_price = add_price.drop(columns=["SPY", "SHV"], errors="ignore")
factor_ret = factor_price.pct_change()
factor_ret = factor_ret.loc[common_idx]
factor_ret.head()

,IEF,IYR,GLD,USO,XBTUSD BGN Curncy
date,,,,,
2015-07-03,NaN,NaN,NaN,NaN,NaN
2015-07-10,-0.000286,0.016816,-0.002416,-0.066561,0.125029
2015-07-17,0.004014,0.008403,-0.025473,-0.038483,-0.030857
2015-07-24,0.007043,-0.005914,-0.030373,-0.056504,0.035541
2015-07-31,0.006049,0.011900,-0.003987,-0.027449,-0.012042


In [94]:
alpha_lo_sec, beta_lo_sec, r2_lo_sec, m_lo_sec = run_lfd(
    carry_ret, factor_ret
)

alpha_ls_sec, beta_ls_sec, r2_ls_sec, m_ls_sec = run_lfd(
    ls_ret, factor_ret
)

factor_beta = pd.DataFrame({
    "LO": beta_lo_sec,
    "LS": beta_ls_sec
})

factor_beta


,LO,LS
IEF,0.301775,0.007604
IYR,-0.159863,-0.058450
GLD,-0.041631,-0.016525
USO,0.047575,0.045882
XBTUSD BGN Curncy,0.026360,-0.002960


In [95]:
factor_stats = pd.DataFrame({
    "Alpha (ann.)": [alpha_lo_sec, alpha_ls_sec],
    "R²": [r2_lo_sec, r2_ls_sec]
}, index=["LO", "LS"])

factor_stats


,Alpha (ann.),R²
LO,0.103318,0.061621
LS,0.028947,0.045882


In [96]:
factor_std = factor_ret.std()

std_exposure = pd.DataFrame({
    "LO": beta_lo_sec * factor_std,
    "LS": beta_ls_sec * factor_std
})

max_lo_sector = std_exposure["LO"].abs().idxmax()
max_ls_sector = std_exposure["LS"].abs().idxmax()

std_exposure, max_lo_sector, max_ls_sector


(                         LO        LS
 IEF                0.002699  0.000068
 IYR               -0.004857 -0.001776
 GLD               -0.000826 -0.000328
 USO                0.002601  0.002508
 XBTUSD BGN Curncy  0.002502 -0.000281,
 'IYR',
 'USO')

<span style="color: blue;">
ANS:

The carry strategy is both neutral to sectors. IYR has the largest exposure, thought it is still small.

In [97]:
mag7 = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'TSLA']
mag_px = df.loc[:, (mag7, "PX_LAST")]
mag_ret = mag_px.pct_change().mean(axis=1)
mag_ret = mag_ret.loc[common_idx]
mag_ret.name = "MAG"
mag_ret.head()


date
2015-07-03         NaN
2015-07-10   -0.012956
2015-07-17    0.085624
2015-07-24   -0.009611
2015-07-31    0.001059
Name: MAG, dtype: float64

In [98]:
X_mag = pd.concat([spy_ret, mag_ret], axis=1)
X_mag.columns = ["SPY", "MAG"]

alpha_lo_mag, beta_lo_mag, r2_lo_mag, m_lo_mag = run_lfd(
    carry_ret, X_mag
)

alpha_ls_mag, beta_ls_mag, r2_ls_mag, m_ls_mag = run_lfd(
    ls_ret, X_mag
)

mag_summary = pd.DataFrame({
    "LO Alpha (ann.)": [alpha_lo_mag],
    "LO β_SPY": [beta_lo_mag["SPY"]],
    "LO β_MAG": [beta_lo_mag["MAG"]],
    "LO R²": [r2_lo_mag],
    
    "LS Alpha (ann.)": [alpha_ls_mag],
    "LS β_SPY": [beta_ls_mag["SPY"]],
    "LS β_MAG": [beta_ls_mag["MAG"]],
    "LS R²": [r2_ls_mag]
})

mag_summary



,LO Alpha (ann.),LO β_SPY,LO β_MAG,LO R²,LS Alpha (ann.),LS β_SPY,LS β_MAG,LS R²
0,0.002618,0.751736,0.010803,0.653896,0.00791,0.070806,0.00868,0.016553


<span style="color: blue;">

ANS:

For the LO strategy, the SPY beta is 0.75 and the MAG beta is near zero (0.01), with a high R² of 0.65. This shows that the LO strategy is mainly driven by overall market exposure, with negligible direct exposure to the Magnificent Seven.

For the LS strategy, both SPY and MAG betas are close to zero, and the R² is only 0.02, indicating that the LS strategy is largely independent of both market and MAG factors. Its return is therefore primarily driven by non-market sources.

Overall, adding the MAG factor does not materially improve the explanatory power of the model.


In [99]:
# 3

window = 52 * 5

hedge_beta_lo = pd.DataFrame(index=sector_ret.index, columns=sector_ret.columns)

for t in range(window, len(sector_ret)):
    Y = carry_ret.iloc[t-window:t]
    X = sm.add_constant(sector_ret.iloc[t-window:t])
    model = sm.OLS(Y, X, missing="drop").fit()
    hedge_beta_lo.iloc[t] = model.params.drop("const")

hedge_beta_ls = pd.DataFrame(index=sector_ret.index, columns=sector_ret.columns)

for t in range(window, len(sector_ret)):
    Y = ls_ret.iloc[t-window:t]
    X = sm.add_constant(sector_ret.iloc[t-window:t])
    model = sm.OLS(Y, X, missing="drop").fit()
    hedge_beta_ls.iloc[t] = model.params.drop("const")



In [100]:
hedge_beta_lo = hedge_beta_lo.shift(1)
hedge_beta_ls = hedge_beta_ls.shift(1)

hedged_lo_ret = carry_ret - (hedge_beta_lo * sector_ret).sum(axis=1)
hedged_lo_ret = hedged_lo_ret.dropna()

hedged_ls_ret = ls_ret - (hedge_beta_ls * sector_ret).sum(axis=1)
hedged_ls_ret = hedged_ls_ret.dropna()



In [101]:
# 3.1

def univariate_stats(r, freq=52):
    return pd.Series({
        "Mean (ann.)": r.mean() * freq,
        "Vol (ann.)": r.std() * np.sqrt(freq),
        "Sharpe": (r.mean() / r.std()) * np.sqrt(freq),
        "Min": r.min(),
        "Max": r.max()
    })

stats_hedged = pd.concat([
    univariate_stats(hedged_lo_ret).rename("Hedged LO"),
    univariate_stats(hedged_ls_ret).rename("Hedged LS")
], axis=1)

stats_hedged


,Hedged LO,Hedged LS
Mean (ann.),0.112963,0.021268
Vol (ann.),0.172117,0.099590
Sharpe,0.656313,0.213550
Min,-0.145055,-0.051743
Max,0.185052,0.106062


In [ ]:
# 3.2

common_idx_hedged_lo = hedged_lo_ret.index.intersection(factor_ret.index)
common_idx_hedged_ls = hedged_ls_ret.index.intersection(factor_ret.index)

Y_lo = hedged_lo_ret.loc[common_idx_hedged_lo]
X_lo = factor_ret.loc[common_idx_hedged_lo]
Y_lo = pd.to_numeric(Y_lo, errors="coerce")

alpha_hlo, beta_hlo, r2_hlo, m_hlo = run_lfd(Y_lo, X_lo)

Y_ls = hedged_ls_ret.loc[common_idx_hedged_ls]
X_ls = factor_ret.loc[common_idx_hedged_ls]
Y_ls = pd.to_numeric(Y_ls, errors="coerce")

alpha_hls, beta_hls, r2_hls, m_hls = run_lfd(Y_ls, X_ls)

hedged_factor_beta = pd.DataFrame({
    "Hedged LO": beta_hlo,
    "Hedged LS": beta_hls
})

hedged_factor_stats = pd.DataFrame({
    "Alpha (ann.)": [alpha_hlo, alpha_hls],
    "R²": [r2_hlo, r2_hls]
}, index=["Hedged LO", "Hedged LS"])


hedged_factor_stats



,Alpha (ann.),R²
Hedged LO,0.099939,0.03817
Hedged LS,0.024913,0.02200


In [111]:
hedged_factor_beta

,Hedged LO,Hedged LS
IEF,0.435167,0.085176
IYR,-0.097569,-0.033827
GLD,-0.048955,-0.015836
USO,0.028919,0.037118
XBTUSD BGN Curncy,0.024076,-0.001699


In [ ]:
# 4